# OCR with Pytesseract and OpenCV

In [1]:
import cv2
import pytesseract
import numpy as np
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

In [2]:

# Adding custom options
custom_config = r'--oem 3 --psm 6'
#pytesseract.image_to_string(img, config=custom_config)

# Preprocessing for Tesseract

In [3]:



# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 


In [4]:
img = cv2.imread('disjB.png')

gray = get_grayscale(img)
img = thresholding(gray)
#img = deskew(img)
#opening = opening(gray)
#canny = canny(gray)
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

# Getting boxes around text

In [ ]:


img = cv2.imread('image.jpg')
img = get_grayscale(img)

cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

#h, w, c = img.shape
h, w = img.shape
boxes = pytesseract.image_to_boxes(img) 

for b in boxes.splitlines():
    b = b.split(' ')
    img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

Error: Kernel is dead

In [ ]:
from pytesseract import Output

img = cv2.imread('image.jpg')
img = get_grayscale(img)

d = pytesseract.image_to_data(img, output_type=Output.DICT)
print(d.keys())

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [ ]:
n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        #crop_img = img[y:y+h, x:x+w]
        #cv2.imshow("cropped", crop_img)
        #cv2.waitKey(0)
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

# Text template matching

In [ ]:
import re
import cv2
import pytesseract
from pytesseract import Output

img = cv2.imread('image.jpg')
img = get_grayscale(img)

d = pytesseract.image_to_data(img, output_type=Output.DICT)
keys = list(d.keys())

date_pattern = '^(0[1-9]|[12][0-9]|3[01])/(0[1-9]|1[012])/(19|20)\d\d$'

n_boxes = len(d['text'])

for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
    	if re.match(date_pattern, d['text'][i]):
	        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
	        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

233


-1